<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Source" data-toc-modified-id="Data-Source-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Source</a></span></li><li><span><a href="#Imports-and-Update-The-Latest-Data" data-toc-modified-id="Imports-and-Update-The-Latest-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports and Update The Latest Data</a></span></li><li><span><a href="#Historical-air-4-Thai-Data" data-toc-modified-id="Historical-air-4-Thai-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Historical air 4 Thai Data</a></span></li><li><span><a href="#Power-Plants" data-toc-modified-id="Power-Plants-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Power Plants</a></span></li><li><span><a href="#Weather-Files" data-toc-modified-id="Weather-Files-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Weather Files</a></span><ul class="toc-item"><li><span><a href="#Assemble-from-raw-data" data-toc-modified-id="Assemble-from-raw-data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Assemble from raw data</a></span></li><li><span><a href="#Update-weather-all-cities" data-toc-modified-id="Update-weather-all-cities-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Update weather all cities</a></span></li><li><span><a href="#Add-New-Weather-Stations-from-URL" data-toc-modified-id="Add-New-Weather-Stations-from-URL-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Add New Weather Stations from URL</a></span></li></ul></li><li><span><a href="#Merge-Weather-Data-from-OpenWeatherMap" data-toc-modified-id="Merge-Weather-Data-from-OpenWeatherMap-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Merge Weather Data from OpenWeatherMap</a></span><ul class="toc-item"><li><span><a href="#Fix-Weather-Units" data-toc-modified-id="Fix-Weather-Units-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Fix Weather Units</a></span></li></ul></li><li><span><a href="#Holiday-In-Thailand" data-toc-modified-id="Holiday-In-Thailand-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Holiday In Thailand</a></span></li><li><span><a href="#Hotspots-Data" data-toc-modified-id="Hotspots-Data-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Hotspots Data</a></span><ul class="toc-item"><li><span><a href="#Fire-For-Chiang-Mai" data-toc-modified-id="Fire-For-Chiang-Mai-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Fire For Chiang Mai</a></span></li><li><span><a href="#Impove-Speed-of-Files-reading" data-toc-modified-id="Impove-Speed-of-Files-reading-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Impove Speed of Files reading</a></span></li></ul></li><li><span><a href="#Data-From-Hanoi-US-Embassy" data-toc-modified-id="Data-From-Hanoi-US-Embassy-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Data From Hanoi US Embassy</a></span></li><li><span><a href="#Vietnam-EPA" data-toc-modified-id="Vietnam-EPA-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Vietnam EPA</a></span></li></ul></div>

# Data Source 
1. Berekely Earth 'http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/'
2. Screaped Air4Thai Data 'http://air4thai.pcd.go.th/webV2/history/'
3. CDC Data 'https://www.cmuccdc.org/download_json/'
4. Old Air4Thai data from Thailand EPA

# Imports and Update The Latest Data

Update pollution data from 3 sources and update weather data 

In [10]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.read_data import *
from src.data.fire_data import *
from src.data.vn_data import *
from src.data.weather_data import *
from src.gen_functions import *
from src.features.dataset import Dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
b_folder='../data/pm25/'
a4th_folder='../data/air4thai_hourly/'
cm_folder ='../data/cm_proc/'
cdc_folder = '../data/cdc_data/'
aqm_folder = '../data/aqm_hourly2/'
aqm_folder2 = '../data/aqm_hourly3/'
w_folder = '../data/weather_cities/'

In [18]:
main(cdc_data=True, build_json=True)

download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/
download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Viet_Nam/
download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Indonesia/
download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Myanmar/
download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Singapore/
download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Laos/
download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Cambodia/
100% [............................................................................] 449726 / 449726download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Malaysia/
100% [..........................


-1 / unknowndownload data from Chiang Mai University Project (CDC)
number of stations 496



Update weather data for all cities


update weather data for  Mueang Chiang Mai
updateing file: ../data/weather_cities/Mueang_Chiang_Mai.csv
missing date 0
update weather data for  Soc Son
updateing file: ../data/weather_cities/Soc_Son.csv
missing date 3



update weather data for  Soc Son
updateing file: ../data/weather_cities/Soc_Son.csv
missing date 0
update weather data for  Bangkok
updateing file: ../data/weather_cities/Bangkok.csv
missing date 3


...


update weather data for  Kunming
updateing file: ../data/weather_cities/Kunming.csv
missing date 3



update weather data for  East Jakarta
updateing file: ../data/weather_cities/East_Jakarta.csv
missing date 3



update weather data for  Mueang Nakhon Si Thammarat
updateing file: ../data/weather_cities/Mueang_Nakhon_Si_Thammarat.csv
missing date 3



update weather data for  Hai Chau
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 3



update weather data for  Chaloem Phra Kiat
updateing file: ../data/weather_cities/Chaloem_Phra_Kiat.csv
missing date 3


In [19]:
main(cdc_data=False, build_json=False)

download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/
100% [............................................................................] 882554 / 882554download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Viet_Nam/
100% [............................................................................] 845829 / 845829download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Indonesia/
100% [................................................................................] 6073 / 6073download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Myanmar/
100% [............................................................................] 174233 / 174233download Berkeley PM2.5 data for all files in http://berkeleyearth.lbl.gov/air-quality/maps/cities/Singapore/
download Berkeley PM2.5 data for all files in http://berke


-1 / unknownUpdate weather data for all cities


update weather data for  Mueang Chiang Mai
updateing file: ../data/weather_cities/Mueang_Chiang_Mai.csv
missing date 0
update weather data for  Soc Son
updateing file: ../data/weather_cities/Soc_Son.csv
missing date 1



update weather data for  Soc Son
updateing file: ../data/weather_cities/Soc_Son.csv
missing date 0
update weather data for  Bangkok
updateing file: ../data/weather_cities/Bangkok.csv
missing date 1



update weather data for  Mueang Chiang Rai
updateing file: ../data/weather_cities/Mueang_Chiang_Rai.csv
missing date 0
update weather data for  Mueang Tak
updateing file: ../data/weather_cities/Mueang_Tak.csv
missing date 1


...


update weather data for  Kunming
updateing file: ../data/weather_cities/Kunming.csv
missing date 1



update weather data for  East Jakarta
updateing file: ../data/weather_cities/East_Jakarta.csv
missing date 1



update weather data for  Mueang Nakhon Si Thammarat
updateing file: ../data/weather_cities/Mueang_Nakhon_Si_Thammarat.csv
missing date 1



update weather data for  Hai Chau
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 1



update weather data for  Chaloem Phra Kiat
updateing file: ../data/weather_cities/Chaloem_Phra_Kiat.csv
missing date 1


In [4]:
download_cdc_data(station_url='https://www.cmuccdc.org/api/ccdc/stations', 
                  dl_url= 'https://www.cmuccdc.org/download_json/', 
                  data_folder='../data/cdc_data/')

  0%|                                                                                                             | 0/404 [00:00<?, ?it/s]

number of stations 430


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 404/404 [39:15<00:00,  5.83s/it]


In [ ]:
update_last_air4Thai(url='http://air4thai.pcd.go.th/webV2/history/',data_folder='../data/air4thai_hourly/')

0it [00:00, ?it/s]

['02t', '03t', '05t', '08t', '10t', '11t', '12t', '13t', '14t', '16t', '17t', '18t', '19t', '20t', '21t', '22t', '24t', '25t', '26t', '27t', '28t', '29t', '30t', '31t', '32t', '33t', '34t', '35t', '36t', '37t', '38t', '39t', '40t', '41t', '42t', '43t', '44t', '46t', '47t', '50t', '52t', '53t', '54t', '57t', '58t', '59t', '60t', '61t', '62t', '63t', '67t', '68t', '69t', '70t', '71t', '72t', '73t', '74t', '75t', '76t', '77t', '79t', '80t', '81t', '82t', '83t', '84t', 'm1', 'm3', 'm4', 'm8', 'm9', 'o10', 'o20', 'o22', 'o23', 'o24', 'o25', 'o26', 'o27', 'o28', 'o29']


44it [13:48, 18.85s/it]

In [61]:
# update weather data 
city_names = [
        'Mueang Chiang Mai',
        'Soc Son',
        'Bangkok',
        'Mueang Chiang Rai',
        'Mueang Tak',
        'Yangon',
        'Tada-U',
        'Sikhottabong',
        'Luang Prabang District',
        'Kunming', 'East Jakarta', 
        'Mueang Nakhon Si Thammarat', 
        'Hai Chau', 'Chaloem Phra Kiat' ]
    
#city_names = ['Mueang Chiang Mai']
weather_station_info = find_weather_stations(city_names, weather_json_file=w_folder + 'weather_station_info.json')
len(weather_station_info)

for city_json in tqdm_notebook(weather_station_info):
    print('update weather data for ', city_json['city_name'])
    start_date = datetime(2020, 8, 22)
    end_date = datetime.now() - timedelta(days=1)
    update_weather(
            city_json,
            data_folder=w_folder,
            start_date=start_date,
            end_date=end_date)

update weather data for  Mueang Chiang Mai
updateing file: ../data/weather_cities/Mueang_Chiang_Mai.csv
missing date 0
update weather data for  Soc Son
updateing file: ../data/weather_cities/Soc_Son.csv
missing date 1



update weather data for  Soc Son
updateing file: ../data/weather_cities/Soc_Son.csv
missing date 0
update weather data for  Bangkok
updateing file: ../data/weather_cities/Bangkok.csv
missing date 1



update weather data for  Mueang Chiang Rai
updateing file: ../data/weather_cities/Mueang_Chiang_Rai.csv
missing date 1



update weather data for  Mueang Tak
updateing file: ../data/weather_cities/Mueang_Tak.csv
missing date 1



update weather data for  Yangon
updateing file: ../data/weather_cities/Yangon.csv
missing date 1


...


update weather data for  Kunming
updateing file: ../data/weather_cities/Kunming.csv
missing date 1



update weather data for  East Jakarta
updateing file: ../data/weather_cities/East_Jakarta.csv
missing date 1



update weather data for  Mueang Nakhon Si Thammarat
updateing file: ../data/weather_cities/Mueang_Nakhon_Si_Thammarat.csv
missing date 1



update weather data for  Hai Chau
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 1



update weather data for  Chaloem Phra Kiat
updateing file: ../data/weather_cities/Chaloem_Phra_Kiat.csv
missing date 1


# Historical air 4 Thai Data

In [7]:
# load stations information for air4 Thai
station_info_file = aqm_folder + 'stations_locations.json'
with open(station_info_file, 'r',encoding="utf8") as f:
    station_info = json.load(f)
station_info = station_info['stations']

# find stations in Chiangmai and parase that files
cm_station_ids = []
cm_station_info = []
for i, stations in enumerate(station_info):
    if 'Chiang Mai' in stations['areaEN']:
        cm_station_ids.append(stations['stationID'])
        cm_station_info.append(stations)
print(cm_station_ids)

['35t', '36t', 'm9', 'o10', 'o20', 'o22']


In [22]:
# parase historical data 
for station_id in cm_station_ids:
    # find all files that start with this stations
    p = Path(aqm_folder)
    filenames = []
    for i in p.glob('**/*.xlsx'):
        if station_id in i.name:
            filenames.append(str(i))
        
    # if filename exist load that file
    if len(filenames) >0:

        save_filename = aqm_folder + 'process/' + station_id + '.csv'
        print('save filename', save_filename)
        station_data = read_his_xl(filenames[0])
        #print(station_data.head())

        # save the data if the dataframe is not empty
        if len(station_data)> 0:
            station_data.to_csv(save_filename,index=True)

save filename ../data/aqm_hourly2/process/35t.csv
save filename ../data/aqm_hourly2/process/36t.csv


In [10]:
# combine new and old air4Thai data 
gas_cols = ['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5']
for station_id in cm_station_ids:
    try: 
        old_data = pd.read_csv(aqm_folder + 'process/'+station_id + '.csv')
    except:
        old_data = pd.DataFrame()
    else:
        old_data['datetime'] = pd.to_datetime(old_data['datetime'])
        old_data = old_data.set_index('datetime')
        # keep only the gass columns
        old_data = old_data[gas_cols]
    
    new_data = pd.read_csv(a4th_folder + station_id + '.csv',na_values='-').set_index('datetime')
    new_data.columns = [s.split(' (')[0] for s in new_data.columns]
    # keep only the gass columns
    new_data = new_data[gas_cols]
    # concatinate data and save
    data = pd.concat([old_data,new_data])
    filename = cm_folder+station_id + '.csv'
    print(filename)
    data.to_csv(filename)

../data/cm_proc/35t.csv
../data/cm_proc/36t.csv
../data/cm_proc/m9.csv
../data/cm_proc/o10.csv
../data/cm_proc/o20.csv
../data/cm_proc/o22.csv


In [45]:
cm_data1 = pd.read_csv(aqm_folder + 'process/35t.csv')
cm_data1['datetime'] = pd.to_datetime(cm_data1['datetime'])
cm_data1 = cm_data1.set_index('datetime')
# keep only gas columns
cm_data1 = cm_data1[['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5']]

cm_data2 = pd.read_csv(aqm_folder + 'process/36t.csv')
cm_data2['datetime'] = pd.to_datetime(cm_data2['datetime'])
cm_data2 = cm_data2.set_index('datetime')


In [8]:
# process bangkok data
station_ids = ['02t',
 '03t',
 '05t',
 '10t',
 '11t',
 '12t',
 '50t',
 '52t',
 '53t',
 '54t',
 '59t',
 '61t']

good_stations = []
bad_stations = []
for ids in station_ids:
    path = Path(aqm_folder)
    files = [*path.rglob(f'({ids})*.xlsx')]
    try:
        df = read_his_xl(files[0])
    except:
        bad_stations.append(ids)
    else:
        good_stations.append(ids)
        df.to_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/aqm_hourly2/process/'+ids+'.csv')
        
print(good_stations)
print(bad_stations)

     date  hour   CO  NO NO2 SO2  NOX      Pressure           Rain  \
0  960101   100  2.6  51  32   5   83                                
1  960101   200  2.8  71  30   6  101                                
2  960101   300  2.8  75  29   6  104                                
3  960101   400  2.5  52  25   5   78                                
4  960101   500  1.4  17  21   5   39                                

        Rel hum          Temp      Wind dir     Wind speed            CO8  
0                                                                          
1                                                                    2.45  
2                                                                 2.25714  
3                                                                  2.1125  
4                                                                   1.975  
    date  hour        CO  NO            O3 NO2 SO2 NOX PM10 PM2.5 Pressure  \
0  30101   100      1.73  16                3

     date  hour   CO NO2 SO2  O3 PM10 PM2.5
0  170101   100    1  18   3  40   26   NaN
1  170101   200  0.9  21   4  34   21   NaN
2  170101   300  0.9  19   3  35   18   NaN
3  170101   400  0.9  17   3  35   16   NaN
4  170101   500  0.9  17   3  34   16   NaN
     date  hour             CO            O3           NO2            SO2  \
0  960101   100                                                             
1  960101   200                                                             
2  960101   300                                                             
3  960101   400                                                             
4  960101   500                                                             

           PM10  
0                
1                
2                
3                
4                
    date  hour   CO O3 NO2 SO2 PM10
0  30101   100  1.5  5  47   6   89
1  30101   200  1.5  5  44   2   49
2  30101   300  1.3  5  36   1   44
3  30101   400  1.3  

     date  hour  CO  NO2 PM10 PM2.5
0  170601   100 NaN  NaN  NaN   NaN
1  170601   200 NaN  NaN  NaN   NaN
2  170601   300 NaN  NaN  NaN   NaN
3  170601   400 NaN  NaN  NaN   NaN
4  170601   500 NaN  NaN  NaN   NaN
     date  hour             CO            O3           NO2            SO2  \
0  960101   100                                                             
1  960101   200                                                             
2  960101   300                                                             
3  960101   400                                                             
4  960101   500                                                             

           PM10  
0                
1                
2                
3                
4                
    date  hour   CO O3 NO2 SO2 PM10
0  30101   100  3.9  3  47  12   51
1  30101   200  4.7  3  46  12   50
2  30101   300  2.1  3  34   4   58
3  30101   400  2.7  3  33   8   69
4  30101   500  3.5  4  37  10   

# Power Plants

In [92]:
url = 'https://th.wikipedia.org/wiki/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%8A%E0%B8%B7%E0%B9%88%E0%B8%AD%E0%B9%82%E0%B8%A3%E0%B8%87%E0%B9%84%E0%B8%9F%E0%B8%9F%E0%B9%89%E0%B8%B2%E0%B9%83%E0%B8%99%E0%B9%84%E0%B8%97%E0%B8%A2'
url = 'https://en.wikipedia.org/wiki/List_of_power_stations_in_Thailand'
table_list = pd.read_html(url)
len(table_list)
p_folder = '../data/power_plants/'
for i, table in enumerate(table_list):
    table.to_csv(p_folder + f'table_eng{i}.csv',index=False)

# Weather Files

In [55]:
folders = glob('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/*/')
len(folders)

9

In [92]:
for folder in folders:
    city_name = Path(folder).name
    parent_folder = Path(folder).parent
    w_files = glob(folder + '/*.csv')
    print(city_name, len(w_files))
    filename = str(Path(folders[0]).parent) + '/' + city_name + '.csv'
    
    # concatenate all files 
    df_all = pd.DataFrame()
    for file in w_files:
        try:
            df = pd.read_csv(file)
            
        except:
            pass
        else:
            df_all = pd.concat([df_all, df], ignore_index=True)
    
    # drop missing value 
    df_all['datetime'] = pd.to_datetime(df_all['date'])
    df_all.drop('date',axis=1, inplace=True)
    df_all = df_all.sort_values('datetime')
    df_all = df_all.drop_duplicates('datetime', ignore_index=True)
    
    # save file
    df_all.to_csv(filename,index=False)

bkk 13
chiang-mai 36
chiang-rai 25
kungming 24
luang-prabang 24
sikhottabong 39
tada_u 24
tak 26
yangong 29


In [89]:
# find missing date for chiang-mai data 
df_all = pd.read_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai.csv')
df_all['datetime'] = pd.to_datetime(df_all['datetime'] )
# find exisiting date 
ex_date = df_all['datetime'].dt.strftime('%Y-%m-%d').unique()
ex_date = set(ex_date)
# calculate the datelist 
start_date = datetime(2000, 10, 1)
stop_date = datetime.now()
date_range = pd.date_range(start_date, stop_date).strftime('%Y-%m-%d')
missing_date = list(set(date_range).difference(ex_date))

In [87]:
with open('../data/weather_cities/weather_station_info.json','r') as f:
    station_dict_list = json.load(f)
   
i = 0 
city_json = station_dict_list[i]
print(city_json)

{'city_name': 'Mueang Chiang Mai', 'province': 'Chiang Mai', 'country': 'Thailand', 'station_name': 'Chiang Mai International Airport Station', 'specific_url': 'th/mueang-chiang-mai/', 'latitude': '18.8 °N', 'longitude': '98.97 °E'}


In [90]:
 bad_date_df = scrape_weather(city_json, date_range=missing_date, data_folder='C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai/')

C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai/mueang-chiang-mai_weather.csv 2000-10-23


..\src\data\dl_weather.py:20: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 20 of the file ..\src\data\dl_weather.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(innerhtml)


In [97]:
# weather data 
filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai.csv'
wea = pd.read_csv(filename)
wea['datetime']  = pd.to_datetime(wea['datetime'])
# roud datetiem to whole 30 mins 
wea['datetime'] = wea['datetime'].dt.round('30T')

dates = wea['datetime'].dropna().dt.date.unique()

# fill in the missing value
new_datetime = pd.date_range(start=dates[0], end=dates[-1], freq='30T') 
new_weather = pd.DataFrame(new_datetime, columns=['datetime'])
new_weather = new_weather.merge(wea, on='datetime',how='left')
print(new_weather.shape)

# remove strange T reading
lowest_t = 5 
idx = new_weather[new_weather['Temperature(C)']< lowest_t].index
new_weather.loc[idx,['Temperature(C)','Dew Point(C)','Humidity(%)']] = np.nan

highest_t = 60
idx = new_weather[new_weather['Temperature(C)']> highest_t].index
new_weather.loc[idx,['Temperature(C)','Dew Point(C)','Humidity(%)']] = np.nan

new_weather = new_weather.fillna(method='ffill',limit=12)
new_weather = new_weather.fillna(method='bfill',limit=12)
new_weather = new_weather.set_index('datetime')
new_weather = new_weather.dropna(how='all').reset_index()
new_weather.to_csv(filename,index=False)

(333126, 11)


## Assemble from raw data 

In [7]:
# weather folder
w_folder = '../data/weather_cities/'

In [8]:
# extract station information
city_names = ['Mueang Chiang Mai', 'Mueang Chiang Rai', 'Mueang Tak','Bangkok','Yangon', 'Tada-U', 'Sikhottabong', 'Luang Prabang District','Kunming']
weather_station_info = find_weather_stations(city_names, weather_json_file=w_folder+'weather_station_info.json')
len(weather_station_info)

9

In [11]:
# concatenate raw data files
for city_json in weather_station_info: 
    # read existing file 
    city_name = ('_').join(city_json['city_name'].split(' '))
    current_filename = w_folder + city_name + '.csv'
    print(current_filename)
    
    # locate file in the city folder
    files = glob(w_folder + city_name + '/*.csv')
    print(len(files))
    
    weather_all = pd.DataFrame()
    for file in files:
        try:
            df = pd.read_csv(file)
        except:
            pass
        else:
            if 'date'== df.columns[-1]:
                df.columns = df.columns.str.replace('date','datetime')
            weather_all = pd.concat([weather_all, df],ignore_index=True)
        
    print(weather_all.columns)
    weather_all.to_csv(current_filename, index=False)
    

../data/weather_cities/Mueang_Chiang_Mai.csv
36
Index(['Time', 'Temperature(C)', 'Dew Point(C)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Condition', 'datetime'],
      dtype='object')
../data/weather_cities/Mueang_Chiang_Rai.csv
25
Index(['Time', 'Temperature(C)', 'Dew Point(C)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Condition', 'datetime'],
      dtype='object')
../data/weather_cities/Mueang_Tak.csv
26
Index(['Time', 'Temperature(C)', 'Dew Point(C)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Condition', 'datetime'],
      dtype='object')
../data/weather_cities/Bangkok.csv
13
Index(['Time', 'Temperature(C)', 'Dew Point(C)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Condition', 'datetime', 'Temperature(F)', 'Dew Point(F)',
      

In [12]:
# fix the unit of windspeed 
files = glob(w_folder + '/*.csv')
for file in files:
    print(file)
    df = pd.read_csv(file)
    df[['Wind Speed(mph)', 'Wind Gust(mph)']] = df[['Wind Speed(mph)', 'Wind Gust(mph)']]*1.60934
    df[['Wind Speed(mph)', 'Wind Gust(mph)']] = df[['Wind Speed(mph)', 'Wind Gust(mph)']].round(0)
    df.columns = df.columns.str.replace('mph', 'kmph')
    df.to_csv(file,index=False)

../data/weather_cities\Bangkok.csv
../data/weather_cities\Kunming.csv
../data/weather_cities\Luang_Prabang_District.csv
../data/weather_cities\Mueang_Chiang_Mai.csv
../data/weather_cities\Mueang_Chiang_Rai.csv
../data/weather_cities\Mueang_Tak.csv
../data/weather_cities\Sikhottabong.csv
../data/weather_cities\Tada-U.csv
../data/weather_cities\Yangon.csv


In [188]:
# drop duplicate datetime 
files = glob(w_folder + '/*.csv')
for file in files:
    df = pd.read_csv(file)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime')
    df = df.drop_duplicates('datetime')
    df.to_csv(file,index=False)

In [17]:
# Fill the missing value in raw Weather Data 
for file in files:
    df = pd.read_csv(file)
    df = fill_missing_weather(df,limit=12)
    df.to_csv(file, index=False)

## Update weather all cities

In [240]:
i = 0
start_date = datetime(2020, 1, 1)
end_date = datetime.now()
city_json = weather_station_info[i]
# read existing file 
city_name = ('_').join(city_json['city_name'].split(' '))
current_filename = w_folder + city_name + '.csv'
print(current_filename)

# obtain a list of existed dates
wea = pd.read_csv(current_file)
wea['datetime'] = pd.to_datetime(wea['datetime'])
# find exisiting date 
ex_date = wea['datetime'].dt.strftime('%Y-%m-%d').unique()
ex_date = set(ex_date)

# calculate the missing dates 
date_range = pd.date_range(start_date, end_date).strftime('%Y-%m-%d')
missing_date = list(set(date_range).difference(ex_date))
missing_date.sort()
len(missing_date)

../data/weather_cities/Mueang_Chiang_Mai.csv


15

In [12]:
# extract station information
city_names = ['Mueang Chiang Mai', 'Mueang Chiang Rai', 'Mueang Tak','Bangkok','Yangon', 'Tada-U', 'Sikhottabong', 'Luang Prabang District','Kunming']
weather_station_info = find_weather_stations(city_names, weather_json_file=w_folder+'weather_station_info.json')
len(weather_station_info)

9

In [13]:
for city_json in tqdm(weather_station_info[:2]):
    
    start_date = datetime(2020,1,1)
    end_date = datetime.now()
    update_weather(city_json, data_folder=w_folder, start_date=start_date, end_date=end_date)

  0%|                                                                                                                                 | 0/2 [00:00<?, ?it/s]

updateing file: ../data/weather_cities/Mueang_Chiang_Mai.csv
missing date 4


 50%|████████████████████████████████████████████████████████████                                                            | 1/2 [03:09<03:09, 189.81s/it]

updateing file: ../data/weather_cities/Mueang_Chiang_Rai.csv
missing date 4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [06:14<00:00, 187.46s/it]


In [3]:
# extract station information
city_names = ['Hai Chau']
weather_station_info = find_weather_stations(city_names, weather_json_file=w_folder+'weather_station_info.json')
len(weather_station_info)
city_json = weather_station_info[0]
print(city_json)

{'city_name': 'Hai Chau', 'province': 'Da Nang', 'country': 'Vietnam', 'station_name': 'Da Nang International Airport Station', 'specific_url': 'vn/hai-chau/', 'latitude': '16.04 °N', 'longitude': '108.2 °E'}


In [4]:
year_list = [2020]
month_list = np.arange(3, 0, -1)

for year, month in product(year_list, month_list):
    start_date = datetime(year, month, 1)
    print(start_date)
    end_date = datetime.now() - timedelta(days=1)
    update_weather(city_json,data_folder=w_folder, start_date=start_date,end_date=end_date)


2020-03-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 31



2020-02-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 29



2020-01-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 31


In [5]:
# add weather data for Hai Chau
year_list = [2019, 2018, 2017, 2016, 2015]
month_list = np.arange(12, 0, -1)

for year, month in product(year_list, month_list):
    start_date = datetime(year, month, 1)
    print(start_date)
    end_date = datetime.now() - timedelta(days=1)
    update_weather(city_json,data_folder=w_folder, start_date=start_date,end_date=end_date)

2019-12-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 31



2019-11-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 30



2019-10-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 31



2019-09-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 30



2019-08-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 31


...


2015-05-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 34



2015-04-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 33



2015-03-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 34



2015-02-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 31



2015-01-01 00:00:00
updateing file: ../data/weather_cities/Hai_Chau.csv
missing date 34


## Add New Weather Stations from URL

In [77]:
station_info

[{'city_name': 'Mueang Chiang Mai',
  'province': 'Chiang Mai',
  'country': 'Thailand',
  'station_name': 'Chiang Mai International Airport Station',
  'specific_url': 'th/mueang-chiang-mai/',
  'latitude': '18.8 °N',
  'longitude': '98.97 °E'},
 {'city_name': 'Mueang Chiang Rai',
  'province': 'Chiang Rai',
  'country': 'Thailand',
  'station_name': 'Chiang Rai Station',
  'specific_url': 'th/mueang-chiang-rai/',
  'latitude': '19.91 °N',
  'longitude': '99.83 °E'},
 {'city_name': 'Mueang Tak',
  'province': 'Tak',
  'country': 'Thailand',
  'station_name': 'Tak Airport Station',
  'specific_url': 'th/mueang-tak/',
  'latitude': '16.87 °N',
  'longitude': '99.13 °E'},
 {'city_name': 'Mueang Mae Hong Son',
  'province': 'Mae Hong Son',
  'country': 'Thailand',
  'station_name': 'Mae Hong Son Airport Station',
  'specific_url': 'th/mueang-mae-hong-son/',
  'latitude': '19.3 °N',
  'longitude': '97.97 °E'},
 {'city_name': 'Mueang Nan',
  'province': 'Nan',
  'country': 'Thailand',
  'st

In [12]:
station_list = ['https://www.wunderground.com/history/daily/th/chaloem-phra-kiat/VTUQ/date/2020-10-21' ] 
add_weather_station(station_list)

add  Chaloem Phra Kiat


# Merge Weather Data from OpenWeatherMap

In [9]:
# weather data 
city_name = 'Chaloem_Phra_Kiat'
final_filename = f'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/{city_name}.csv'
try: 
    wea = pd.read_csv(filename)
    wea['datetime'] = pd.to_datetime(wea['datetime'])
    
except:
    wea = pd.DataFrame()

In [10]:
folder = f'C:/Users/Benny/Documents/Fern/BKK-AQI/data/weather_cities/{city_name}/'
files = glob(folder+'*.csv')
# process data from OpenWeatherMap.org 
wea_df = pd.read_csv(files[0])
wea_df = proc_open_weather(wea_df)

In [11]:
new_wea = pd.concat([wea_df,wea])
new_wea = new_wea.sort_values(['datetime'],ignore_index=True)
new_wea = new_wea.drop_duplicates('datetime')

In [12]:
new_wea[new_wea['datetime'] > '2019-01-01'].tail(10)

,datetime,Time,Temperature(C),Humidity(%),Wind,Wind_Speed(kmph,Pressure(hPa),Precip.(mm),Condition
271582,2020-10-21 21:00:00,09:00 PM,22.37,85,ENE,10.692,1011,0.0,Clouds
271583,2020-10-21 22:00:00,10:00 PM,22.34,85,ENE,10.692,1011,0.0,Clouds
271584,2020-10-21 23:00:00,11:00 PM,22.37,85,ENE,10.692,1011,0.0,Clouds
271585,2020-10-22 00:00:00,12:00 AM,20.48,83,E,11.628,1011,0.0,Clear
271586,2020-10-22 01:00:00,01:00 AM,20.46,83,E,11.628,1011,0.0,Clear
271587,2020-10-22 02:00:00,02:00 AM,20.48,83,E,11.628,1011,0.0,Clear
271588,2020-10-22 03:00:00,03:00 AM,19.42,87,ENE,7.884,1010,0.0,Clouds
271589,2020-10-22 04:00:00,04:00 AM,19.42,87,ENE,7.884,1010,0.0,Clouds
271590,2020-10-22 05:00:00,05:00 AM,20.29,100,N,3.600,1011,0.0,Clouds
271591,2020-10-22 06:00:00,06:00 AM,20.46,100,N,3.600,1012,0.0,Clouds


In [17]:
new_wea.to_csv(final_filename, index=False)

## Fix Weather Units

In [59]:
weather_files = glob(f'C:/Users/Benny/Documents/Fern/BKK-AQI/data/weather_cities/*.csv')

for file in weather_files:
    wea = pd.read_csv(file)
    
    # change columns order 
    wea = wea[['datetime', 'Time', 'Temperature(C)', 'Dew Point(C)', 'Humidity(%)',
       'Wind', 'Wind_Speed(kmph)', 'Wind Gust(kmph)', 'Pressure(in)',
       'Precip.(in)', 'Condition']]
    # change Pressure col 
    col = 'Pressure(in)'
    wea[col] = (wea[col]* 33.8638).round(0)
    #wea.columns = wea.columns.str.replace('in', 'hPa')
    col = 'Precip.(in)'
    wea[col] = (wea[col]* 25.4).round(2)

    wea.columns = ['datetime', 'Time', 'Temperature(C)', 'Dew Point(C)', 'Humidity(%)',
       'Wind', 'Wind_Speed(kmph)', 'Wind Gust(kmph)', 'Pressure(hPa)',
       'Precip.(mm)', 'Condition'] 
    wea.to_csv(file, index=False)

In [3]:
weather_files = glob(f'C:/Users/Benny/Documents/Fern/BKK-AQI/data/weather_cities/*.csv')

In [ ]:
for file in weather_files:
    df = pd.read_csv(weather_files[-1])
    df.columns = df.columns.str.replace(' ', '_')
    df.to_csv(file, index=False)
    

# Holiday In Thailand

https://www.timeanddate.com/holidays/thailand/2001

In [13]:
years = np.arange(2000,2021)

In [14]:
# scrape holiday from all websites
th_holiday = pd.DataFrame()
for year in years:
    url = f'https://www.timeanddate.com/holidays/thailand/{year}'
    df = pd.read_html(url)[0]
    df['year'] = year
    th_holiday = pd.concat([th_holiday,df],ignore_index=True)

In [17]:
th_holiday.columns = ['Date', 'day_of_week','name','type','year']

In [22]:
th_holiday = th_holiday[~th_holiday['Date'].isna()]

In [28]:
th_holiday['date'] = th_holiday['Date'] + ', ' + th_holiday['year'].astype(str)
th_holiday['date'] = pd.to_datetime(th_holiday['date'])

In [30]:
th_holiday.to_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/th_holiday.csv',index=False)

# Hotspots Data 

In [16]:
# load stations information for Chiangmai
station_info_file = aqm_folder + 'stations_locations.json'
with open(station_info_file, 'r',encoding="utf8") as f:
    station_info = json.load(f)
station_info = station_info['stations']

print(station_info[124])

{'stationID': '36t', 'nameTH': 'โรงเรียนยุพราชวิทยาลัย ', 'nameEN': 'Yupparaj Wittayalai School', 'areaTH': 'ต.ศรีภูมิ อ.เมือง, เชียงใหม่', 'areaEN': 'Si Phum, Meuang, Chiang Mai', 'stationType': 'GROUND', 'lat': '18.7909205', 'long': '98.9881062', 'LastUpdate': {'date': '2020-03-27', 'time': '03:00', 'PM25': {'value': '87', 'unit': 'µg/m³'}, 'PM10': {'value': '110', 'unit': 'µg/m³'}, 'O3': {'value': 'N/A', 'unit': 'ppb'}, 'CO': {'value': '1.09', 'unit': 'ppm'}, 'NO2': {'value': '-', 'unit': 'ppb'}, 'SO2': {'value': '2', 'unit': 'ppb'}, 'AQI': {'Level': '4', 'aqi': '192'}}}


In [17]:
# obtain the lat and long in km
lat_km = (merc_y(station_info[124]['lat'])/1E3 ).round()
long_km = (merc_x(station_info[124]['long'])/1E3).round()
print(lat_km,  long_km )

2117.0 11019.0


In [2]:
# file map folder 
m_files = glob('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/fire_map/world_2000-2020/M6/*.csv')
v_files = glob('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/fire_map/world_2000-2020/V1/*.csv')

In [19]:
# keep the file spot with distance 1000 km from the station
distance = 1000 # km

filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_m.csv'


file_all = pd.DataFrame()

for file in tqdm(m_files):
    f = pd.read_csv(file)
     
    # convert lat 
    f['lat_km'] = (f['latitude'].apply(merc_y)/1E3).round().astype(int)
    f['long_km'] = (merc_x(f['longitude'])/1E3).round().astype(int)
    # remove by lat 
    f = f[f['lat_km'] <= (lat_km+1000)]
    f = f[f['lat_km'] >= (lat_km-1000)]
    # remove by long 

    f = f[f['long_km'] <= (long_km+1000)]
    f = f[f['long_km'] >= (long_km-1000)]
     
    file_all = pd.concat([file_all,f],ignore_index=True)
        
        
file_all.to_csv(filename, index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [13:35<00:00, 38.83s/it]


In [20]:
m_filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_m.csv'
process_fire_data(m_filename)


c:\users\benny\pyenv\geo\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


before drop (3901918, 18)
after drop (3829241, 18)


In [40]:
# keep the file spot with distance 1000 km from the station
distance = 1000 # km

filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_v.csv'

fire_all = pd.DataFrame()

for file in v_files:
    f = pd.read_csv(file)
     
    # convert lat 
    f['lat_km'] = (f['latitude'].apply(merc_y)/1E3).round()
    f['long_km'] = (merc_x(f['longitude'])/1E3).round()
    # remove by lat 
    f = f[f['lat_km'] <= (lat_km+1000)]
    f = f[f['lat_km'] >= (lat_km-1000)]
    # remove by long 

    f = f[f['long_km'] <= (long_km+1000)]
    f = f[f['long_km'] >= (long_km-1000)]
    
    fire_all = pd.concat([fire_all,f],ignore_index=True)  
     
        
fire_all.to_csv(filename, index=False)

In [19]:
df = pd.read_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_v.csv')
df.columns

c:\users\benny\pyenv\geo\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (10,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['latitude', 'longitude', 'bright_ti4', 'scan', 'track', 'acq_date',
       'acq_time', 'satellite', 'instrument', 'confidence', 'version',
       'bright_ti5', 'frp', 'type', 'lat_km', 'long_km', 'daynight'],
      dtype='object')

In [20]:
process_fire_data('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_v.csv')

before drop (9092222, 18)
after drop (6925015, 18)


## Fire For Chiang Mai

In [21]:
city_info = {'Country': 'Thailand',
 'City': 'Chiang Mai',
 'City (ASCII)': 'Chiang Mai',
 'Region': 'Chiang Mai',
 'Region (ASCII)': 'Chiang Mai',
 'Population': '200952',
 'Latitude': '18.7904',
 'Longitude': '98.9847',
 'Time Zone': 'Asia/Bangkok'}

x = merc_x(city_info['Longitude'])
y = merc_y(city_info['Latitude'])
stepx = 800E3
stepy = stepx

#fire_v = pd.read_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_v.csv')
m_filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_m.csv'
fire = pd.read_csv(m_filename, dtype={'version': str})
columns_to_drop = ['acq_date','satellite','instrument','version','daynight','bright_t31','type']
fire = fire.drop(columns_to_drop,axis=1)
fire['datetime'] = pd.to_datetime(fire['datetime'])
fire = fire.sort_values('datetime')
fire = fire.set_index('datetime')
# remove the data before '2002-07-04' because there is only one satellite
fire = fire.loc['2002-07-04':]
print(fire.shape)


# add distance columns
fire['distance'] = np.sqrt((fire['lat_km'] - y/1000) **2 + ((fire['long_km'] - x/1000)**2))

# create power column and drop unncessary columns
fire['power'] = fire['scan']*fire['track']*fire['frp']
fire['count'] = 1
fire = fire.drop(['latitude', 'longitude', 'brightness','year','season','acq_time','track','scan','frp'], axis=1)
fire.to_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_m_proc.csv')

(3779468, 10)


KeyError: "['year' 'season'] not found in axis"

## Impove Speed of Files reading

In [6]:
lat_km = 2117.0 
long_km = 11019.0
m_files = glob('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/fire_map/world_2000-2020/M6/*.csv')

In [4]:
%%time

# check the time without threading  
file_all = pd.DataFrame()

for file in tqdm(m_files):
    f = pd.read_csv(file)
     
    # convert lat 
    f['lat_km'] = (f['latitude'].apply(merc_y)/1E3).round().astype(int)
    f['long_km'] = (merc_x(f['longitude'])/1E3).round().astype(int)
    # remove by lat 
    f = f[f['lat_km'] <= (lat_km+1000)]
    f = f[f['lat_km'] >= (lat_km-1000)]
    # remove by long 

    f = f[f['long_km'] <= (long_km+1000)]
    f = f[f['long_km'] >= (long_km-1000)]
     
    file_all = pd.concat([file_all,f],ignore_index=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [16:11<00:00, 44.15s/it]

Wall time: 16min 11s


In [5]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [22]:

def read_fire(file,lat_km, long_km):
   
    f = pd.read_csv(file)
     
    # convert lat 
    f['lat_km'] = (f['latitude'].apply(merc_y)/1E3).round().astype(int)
    f['long_km'] = (merc_x(f['longitude'])/1E3).round().astype(int)
    # remove by lat 
    f = f[(f['lat_km'] <= (lat_km+1000)) & (f['lat_km'] >= (lat_km-1000))]
    # remove by long 
    f = f[(f['long_km'] <= (long_km+1000)) & (f['long_km'] >= (long_km-1000))]
    return f
 

def read_fire_dd(file,lat_km, long_km):
   
    f = dd.read_csv(file)
     
    # convert lat 
    f['lat_km'] = (f['latitude'].apply(merc_y)/1E3).round().astype(int)
    f['long_km'] = (merc_x(f['longitude'])/1E3).round().astype(int)
    # remove by lat 
    f = f[(f['lat_km'] <= (lat_km+1000)) & (f['lat_km'] >= (lat_km-1000))]
    # remove by long 
    f = f[(f['long_km'] <= (long_km+1000)) & (f['long_km'] >= (long_km-1000))]
    return f
    
    

In [24]:
def read_fires(files,lat_km, long_km):
    
    all_fire = pd.DataFrame()
    for file in files:
        if file != None:
            f = pd.read_csv(file)
     
            # convert lat 
            f['lat_km'] = (f['latitude'].apply(merc_y)/1E3).round().astype(int)
            f['long_km'] = (merc_x(f['longitude'])/1E3).round().astype(int)
            # remove by lat 
            f = f[(f['lat_km'] <= (lat_km+1000)) & (f['lat_km'] >= (lat_km-1000))]
            # remove by long 
            f = f[(f['long_km'] <= (long_km+1000)) & (f['long_km'] >= (long_km-1000))]
            all_fire = pd.concat([all_fire, f], ignore_index=True)
    return all_fire

In [15]:
from itertools import zip_longest

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [11]:
%%time
# try with threadpoolExecutor
with ThreadPoolExecutor(max_workers=3) as executor:
    future_list = [executor.submit(read_fire, file, lat_km, long_km) for file in m_files]
    fire_all = pd.DataFrame()
    for future in concurrent.futures.as_completed(future_list):
        df = future.result()
        print(df.shape)

(229176, 17)
(235249, 17)
(214976, 17)
(234913, 17)
(165272, 17)
(288160, 17)
(230523, 17)
(176189, 17)
(276734, 17)
(198856, 17)
(225431, 17)
(276406, 17)
(7802, 17)
(2898, 17)
(195966, 17)
(45055, 17)
(210563, 17)
(198697, 17)
(142868, 17)
(155097, 17)
(49102, 16)
(149221, 17)
Wall time: 13min 16s


In [25]:
%%time

with ThreadPoolExecutor(max_workers=4) as executor:
    future_list = [executor.submit(read_fires, group, lat_km, long_km) for group in grouper(m_files, 4)]
    fire_all = pd.DataFrame()
    for future in concurrent.futures.as_completed(future_list):
        df = future.result()
        print(df.shape)

(251721, 17)
(860144, 17)
(914314, 17)
(204199, 17)
(977427, 17)
(701349, 17)
Wall time: 17min 39s


In [4]:
import dask.dataframe as dd

In [7]:
m_files[0]

'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/fire_map/world_2000-2020/M6\\fire_archive_M6_100871.csv'

In [10]:
df = dd.read_csv(m_files[0])
df['lat_km'] = (df['latitude'].apply(merc_y)/1E3).round().astype(int)
df['long_km'] = (merc_x(df['longitude'])/1E3).round().astype(int)
# remove by lat 
df = df[(df['lat_km'] <= (lat_km+1000)) & (df['lat_km'] >= (lat_km-1000))]
# remove by long 
df = df[(df['long_km'] <= (long_km+1000)) & (df['long_km'] >= (long_km-1000))]

In [11]:
df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,lat_km,long_km
259,23.4521,107.8448,307.5,3.4,1.7,2015-01-01,410,Terra,MODIS,66,6.2,286.6,49.5,D,0,2670,12005
260,23.4631,107.8534,304.2,3.4,1.7,2015-01-01,410,Terra,MODIS,58,6.2,286.7,37.4,D,0,2671,12006
261,21.6968,103.0861,318.7,1.7,1.3,2015-01-01,411,Terra,MODIS,79,6.2,291.9,35.2,D,0,2459,11475
262,21.6998,103.0696,309.5,1.7,1.3,2015-01-01,411,Terra,MODIS,69,6.2,292.6,19.3,D,0,2460,11474
263,22.1135,101.1773,305.3,1.3,1.1,2015-01-01,411,Terra,MODIS,61,6.2,289.2,9.1,D,0,2509,11263


In [18]:
%%time
# try with dask
all_fire = [read_fire_dd(file,lat_km, long_km) for file in m_files]
all_fire = dd.concat(all_fire)
fire_df = all_fire.compute()

Wall time: 12min 57s


In [2]:
from math import sqrt
from joblib import Parallel, delayed
Parallel(n_jobs=2)(delayed(sqrt)(i ** 2) for i in range(10))
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [20]:
%%time
all_fire_list = Parallel(n_jobs=2)(delayed(read_fire)(file, lat_km, long_km) for file in m_files)
fire_df = pd.concat(all_fire_list, ignore_index=True)

Wall time: 7min 36s


# Data From Hanoi US Embassy

In [114]:
wget.download('http://dosairnowdata.org/dos/historical/Hanoi/2020/Hanoi_PM2.5_2020_YTD.csv','../data/us_emb/Hanoi_PM2.5_2020_YTD.csv')

100% [............................................................................] 389775 / 389775

'../data/us_emb/Hanoi_PM2.5_2020_YTD (1).csv'

# Vietnam EPA

In [335]:
%%time
data, station_info_list  = download_vn_data()

  0%|                                                                                                                                | 0/33 [00:00<?, ?it/s]

number of station 33. Takes about 1 mins per station


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [25:34<00:00, 46.49s/it]


the total data has len 796
save file as  C:\Users\Benny\Documents\Fern\aqi_thailand2\data\vn_epa\2020-08-31_19-20.csv
Wall time: 27min 25s


In [11]:
concat_vn_data()

old file length  133470
new file length  34027
